# Project Assignment 1
Enkhzaya, Tamako, Wilson, Chenjun

In [1]:
import pandas as pd
import numpy as np
df_indicators = pd.read_csv('country_indicators.csv')

In [2]:
df_preds = pd.read_csv('test_predictions.csv')

In [3]:
df_preds['transformer_probability_prediction_error'] = np.abs(df_preds['y_true_transformer'].astype(float) - df_preds['y_pred_proba_transformer'])
df_preds['xgboost_probability_prediction_error'] = np.abs(df_preds['y_true_xgboost'].astype(float) - df_preds['y_pred_proba_xgboost'])
df_preds['ffnn_probability_prediction_error'] = np.abs(df_preds['y_true_ffnn'].astype(float) - df_preds['y_pred_proba_ffnn'])

In [4]:
# creating bootstrap confidence interval for ffnn

np.random.seed(130)

n, R = len(df_preds), 1000
ffnn_bootstrap = []
for i in range(R):
    ffnn_bootstrap.append(df_preds.sample(n, replace=True)['ffnn_probability_prediction_error'].mean())

np.percentile(ffnn_bootstrap, [2.5, 97.5])

array([0.40494034, 0.42866273])

In [5]:
threshold = 0.5
df_preds['transformer_classification_performance_outcome'] = None

tmp = df_preds['transformer_classification_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer>threshold)
tmp[TP_pos_pred_correct] = "correct"
TN_neg_pred_correct = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer<=threshold)
tmp[TN_neg_pred_correct] = "correct"
FP_pos_pred_wrong = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer>threshold)
tmp[FP_pos_pred_wrong] = "incorrect"
FN_neg_pred_wrong = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer<=threshold)
tmp[FN_neg_pred_wrong] = "incorrect"

df_preds['transformer_classification_performance_outcome'] = tmp

df_preds['xgboost_classification_performance_outcome'] = None

tmp = df_preds['xgboost_classification_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost>threshold)
tmp[TP_pos_pred_correct] = "correct"
TN_neg_pred_correct = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost<=threshold)
tmp[TN_neg_pred_correct] = "correct"
FP_pos_pred_wrong = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost>threshold)
tmp[FP_pos_pred_wrong] = "incorrect"
FN_neg_pred_wrong = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost<=threshold)
tmp[FN_neg_pred_wrong] = "incorrect"

df_preds['xgboost_classification_performance_outcome'] = tmp

df_preds['ffnn_classification_performance_outcome'] = None

tmp = df_preds['ffnn_classification_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn>threshold)
tmp[TP_pos_pred_correct] = "correct"
TN_neg_pred_correct = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn<=threshold)
tmp[TN_neg_pred_correct] = "correct"
FP_pos_pred_wrong = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn>threshold)
tmp[FP_pos_pred_wrong] = "incorrect"
FN_neg_pred_wrong = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn<=threshold)
tmp[FN_neg_pred_wrong] = "incorrect"

df_preds['ffnn_classification_performance_outcome'] = tmp


In [6]:
import scipy

In [7]:
# creating one-sample hypothesis test for ffnn 

H0 = 0.5

sample = df_preds['ffnn_classification_performance_outcome'].copy()
np.round(scipy.stats.ttest_1samp(sample == "correct", H0).pvalue, 3)



0.0

In [8]:
df = df_preds.merge(df_indicators, left_on='iso3', right_on='iso3', how='inner')

Our null hypothesis was $H_0: \mu = 0.5$ and our alternative hypothesis was $H_1: \mu \neq 0.5$. 
Since p < 0.001, we have very strong evidence against the null hypothesis. 

In [9]:
warning = df[df['fsi_category'] == 'Warning']
other = df[df['fsi_category'] != 'Warning']

In [10]:
# creating two sample bootstrap for warning and other for ffnn
n1, n2, R = len(warning), len(other), 1000

diffs = []

for i in range(R):
    warning_sample = warning.sample(n1, replace=True)['ffnn_probability_prediction_error'].mean()
    other_sample = other.sample(n2, replace=True)['ffnn_probability_prediction_error'].mean()
    diffs.append(warning_sample - other_sample)

np.round(np.percentile(diffs, [2.5, 97.5]), 3)

array([0.001, 0.047])

In [11]:
# creating two sample hypothesis testing
np.round(scipy.stats.ttest_ind(warning['ffnn_probability_prediction_error'], other['ffnn_probability_prediction_error']).pvalue, 3)

0.053

Our null hypothesis was $H_0: \mu_{\text{warning}} = \mu_{\text{others}}$ and our alternative hypothesis was $H_1: \mu_{\text{warning}} \neq \mu_{\text{others}}$. Since our p-value was 0.053, we have weak evidence against the null hypothesis.

In [12]:
# now we compare the difference between ffnn and xgboost on all the data

n, R = len(df_preds), 1000
ffnn_xgboost_diffs = []

for i in range(R):
    sample = df_preds.sample(n, replace=True)
    ffnn_sample = sample['ffnn_probability_prediction_error'].mean()
    xgboost_sample = sample['xgboost_probability_prediction_error'].mean()
    ffnn_xgboost_diffs.append(ffnn_sample - xgboost_sample)

np.round(np.percentile(ffnn_xgboost_diffs, [2.5, 97.5]), 3)

array([-0.05 , -0.008])

In [13]:
# creating two sample hypothesis testing
np.round(scipy.stats.ttest_rel(df_preds['ffnn_probability_prediction_error'], df_preds['xgboost_probability_prediction_error']).pvalue, 3)

0.007

Our null hypothesis was $H_0: \mu_{\text{ffnn}} = \mu_{\text{xgboost}}$ and our alternative hypothesis was $H_1: \mu_{\text{ffnn}} \neq \mu_{\text{xgboost}}$. Since our p-value was 0.007, we have strong evidence against the null hypothesis.

In [14]:
# two sample bootstrap on warning and other for ffnn and xgboost

n1, n2, R = len(warning), len(other), 1000

diffs = []

for i in range(R):
    curr_warning_sample = warning.sample(n1, replace=True)
    curr_other_sample = other.sample(n2, replace=True)

    ffnn_warning_sample = curr_warning_sample['ffnn_probability_prediction_error'].mean()
    ffnn_other_sample = curr_other_sample['ffnn_probability_prediction_error'].mean()

    xgboost_warning_sample = curr_warning_sample['xgboost_probability_prediction_error'].mean()
    xgboost_other_sample = curr_other_sample['xgboost_probability_prediction_error'].mean()
    
    ffnn_diff = ffnn_warning_sample - ffnn_other_sample
    xgboost_diff = xgboost_warning_sample - xgboost_other_sample

    diffs.append(ffnn_diff - xgboost_diff)

np.round(np.percentile(diffs, [2.5, 97.5]), 3)




array([-0.137, -0.055])

In [15]:
# creating two sample hypothesis testing for ffnn and xgboost on warning data
np.round(scipy.stats.ttest_rel(warning['ffnn_probability_prediction_error'], warning['xgboost_probability_prediction_error']).pvalue, 3)


0.0

Our null hypothesis was $H_0: \mu_{\text{ffnn warning}} - \mu_{\text{xgboost warning}} = 0$ and our alternative hypothesis was $H_1: \mu_{\text{ffnn warning}} - \mu_{\text{xgboost warning}} \neq 0$. Since our p-value was 0.000, we have very strong evidence against the null hypothesis.